In [2]:
import random
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Mecab
from googletrans import Translator

In [5]:
# Back Translation Function
def back_translation(text):
    """
    Perform back translation (Korean → English → Korean) for text augmentation.
    """
    translator = Translator()
    try:
        translated = translator.translate(text, src='ko', dest='en').text
        return translator.translate(translated, src='en', dest='ko').text
    except Exception as e:
        print(f"Back translation error: {e}")
        return text  # Fallback to original text if error occurs

# Random Deletion
def random_deletion(sentence, p=0.1):
    words = sentence.split()
    if len(words) == 1:
        return sentence
    return ' '.join([word for word in words if random.uniform(0, 1) > p])

# Random Word Swap
def random_swap(sentence, n=1):
    words = sentence.split()
    for _ in range(n):
        if len(words) > 1:
            idx1, idx2 = random.sample(range(len(words)), 2)
            words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

# Random Insertion
def random_insertion(sentence, n=1):
    words = sentence.split()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        words.insert(idx, words[idx])
    return ' '.join(words)

# AEDA (Punctuation Insertion)
def aeda_augmentation(sentence, num_punct=1):
    punctuations = ['.', ',', '!', '?', ';']
    words = sentence.split()
    for _ in range(num_punct):
        idx = random.randint(0, len(words))
        punct = random.choice(punctuations)
        words.insert(idx, punct)
    return ' '.join(words)

# Augmentation Function
def augment_text_data(text_col, repetition=1):
    """
    Apply text augmentations (EDA, AEDA, Back-Translation) for each input text.
    
    Ensures output length matches input length by selecting only one augmented version per text.
    """
    augmented_data = []
    for text in tqdm(text_col, desc="Augmenting Text"):
        # Store augmented versions
        augmented_versions = []
        
        for _ in range(repetition):
            try:
                eda_augmented = random_deletion(text, p=0.1)
                eda_augmented = random_swap(eda_augmented, n=1)
                eda_augmented = random_insertion(eda_augmented, n=1)

                aeda_augmented = aeda_augmentation(text, num_punct=1)
                back_translated = back_translation(text)

                # Collect augmented versions
                augmented_versions.extend([eda_augmented, aeda_augmented, back_translated])
            except Exception as e:
                print(f"Augmentation error: {e}")
                augmented_versions.append(text)  # Fallback to original text
        
        # Select one augmented version (e.g., the first) to maintain alignment
        augmented_data.append(augmented_versions[0])
    
    return augmented_data


In [6]:
# Load Data
train_data = pd.read_csv('/root/NLP/data/train.csv')

# Sample Data for Augmentation
train_sample = train_data.sample(n=2500, random_state=42) if len(train_data) > 2500 else train_data

# Apply Augmentation
train_sample['augmented'] = augment_text_data(train_sample['dialogue'], repetition=1)

# Save Augmented Data
train_sample.to_csv("/root/NLP/data/augmented_train.csv", index=False)
print("Data augmentation complete!")


Augmenting Text: 100%|██████████| 2500/2500 [21:03<00:00,  1.98it/s]

Data augmentation complete!


In [8]:
import pandas as pd

# Load the augmented dataset
augmented_data = pd.read_csv('/root/NLP/data/augmented_train.csv')

# Check the size of the dataset (number of rows and columns)
print(f"Dataset size: {augmented_data.shape}")

Dataset size: (2500, 5)


In [9]:
# Preview the first few rows to see the data content
print(augmented_data.head())

# Check dataset size
num_rows, num_columns = augmented_data.shape
print(f"Number of rows (sentences): {num_rows}")
print(f"Number of columns: {num_columns}")

         fname                                           dialogue  \
0    train_396  #Person1#: 안녕하세요, 잭 있나요?\n#Person2#: 저입니다.\n#P...   
1    train_247  #Person1#: 실례합니다. 면접에 가려면 어떤 옷을 입어야 하나요?\n#Per...   
2   train_9260  #Person1#: 존, 당신의 동기에 대해 몇 가지 질문을 하고 싶습니다. 당신이...   
3   train_1430  #Person1#: 실례합니다만, 베를린행 비행기가 언제 출발하는지 알려주시겠어요?...   
4  train_10328  #Person1#: 주, 이 코트 어때?\n#Person2#: 이런, 조지. 이 색...   

                                             summary     topic  \
0                 로즈가 잭에게 전화를 걸어 이번 토요일 저녁 식사에 초대한다.  저녁 식사 초대   
1  #Person2#는 #Person1#에게 정장에 넥타이를 착용하고 면접에서 최선을 ...        면접   
2  존은 #Person1#에게 그의 동기, 그의 경영 스타일, 그리고 그의 사업과 개인...       인터뷰   
3  #Person1#은 다음 베를린행 비행기 티켓을 구매하려 한다. #Person2#은...    비행기 정보   
4  조지는 주가 코트를 고르는 것을 도와주고 있다. 그들은 점원에게 더 큰 사이즈를 물...        코트   

                                           augmented  
0  #Person1#: 있나요? 잭 안녕하세요, #Person2#: 저입니다. 저 로즈...  
1  어떤 면접에 가려면 #Person1#: 옷을 입어야 하나요? #Person2#: 정...  
2  존, 당신의 동기에 몇 가지 질문을 